# Projeto ADEL (Análise Dados Estadia em Limeira) - Webscraping das Linhas de Ônibus
Neste notebook, realizarei a extração dos dados das linhas de ônibus em Limeira, SP. Coletarei as localizações dos pontos de cada linha e a distância de cada ponto da faculdade.

In [79]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from geopy.geocoders import Nominatim
from google.colab import files
import geopy
import geopy.distance
import re
import pandas as pd

## Linhas de ônibus

In [2]:
url = 'https://moovitapp.com/index/pt-br/transporte_p%C3%BAblico-lines-Limeira-4161-908539'
response = urlopen(url)
html_linhas_onibus = response.read()
soup_linhas_onibus = BeautifulSoup(html_linhas_onibus, 'html.parser')

In [44]:
linhas_onibus_links = []
linhas_onibus_nomes = []

# Coletar os links de cada linha de ônibus
for linha_link in soup_linhas_onibus.find('div', {'id': 'agency-lines'}).find_all('a'):
  linhas_onibus_links.append(linha_link['href'])

# Coletar os nomes de cada linha de ônibus
for linha_nome in soup_linhas_onibus.find('div', {'id': 'agency-lines'}).find_all('strong'):
  linhas_onibus_nomes.append(linha_nome.get_text())

# Converter em Series
linhas_onibus = pd.DataFrame(
    {
        'linha_nome': linhas_onibus_nomes,
        'linha_link': linhas_onibus_links
    }
)
linhas_onibus

,linha_nome,linha_link
0,01 Jd. Campo Verde / G. Piccinini x Shopping/A...,https://moovitapp.com/index/pt-br/transporte_p...
1,02 Olga Veroni x Geada,https://moovitapp.com/index/pt-br/transporte_p...
2,03 Nova Limeira x N. Sra. das Dores,https://moovitapp.com/index/pt-br/transporte_p...
3,04 Olga Veroni x Jardim Vanessa,https://moovitapp.com/index/pt-br/transporte_p...
4,06 Campo Novo x N. Sra. das Dores,https://moovitapp.com/index/pt-br/transporte_p...
5,08B Atacadão x Terminal,https://moovitapp.com/index/pt-br/transporte_p...
6,08C Santina - Parque Hipólito via Santa Casa,https://moovitapp.com/index/pt-br/transporte_p...
7,09 Tatu (Parronchi) x Terminal,https://moovitapp.com/index/pt-br/transporte_p...
8,10 Terminal x Pinhal (Pires),https://moovitapp.com/index/pt-br/transporte_p...
9,11 Nobre Ville x Village,https://moovitapp.com/index/pt-br/transporte_p...


In [45]:
num_linhas_onibus = linhas_onibus.shape[0]
print(f'Limeira possui {num_linhas_onibus} linhas de ônibus.')

Limeira possui 27 linhas de ônibus.


## Pontos de ônibus

In [74]:
def get_soup(url):
  response = urlopen(url)
  html = response.read()
  return BeautifulSoup(html, 'html.parser')

def get_pontos_addresses(soup):
  pontos = []
  for ponto in soup.find('ul', {'class': 'stops-list bordered'}).find_all('h3'):
    pontos.append(ponto.get_text())
  return pontos

def get_coords(pontos):
  lats = []
  lons = []

  geolocator = Nominatim(user_agent="EstadiaLimeiraApp")

  for ponto in pontos:
    location = geolocator.geocode(f"{ponto}, Limeira, Brazil")
    if location == None:
      coords = input(f"Digite a coordenada de '{ponto}' (x, y): ").strip()
      coords = coords.split(',')
      coords = (float(coords[0]), float(coords[1]))
      lats.append(coords[0])
      lons.append(coords[1])
    else:
      lats.append(location.latitude)
      lons.append(location.longitude)

  return lats, lons

def get_dists(num_pontos, lats, lons):
  # Localização da faculdade
  coords_faculdade = (-22.561883227911103, -47.42418079671464)
  distances = []

  # Calcular as distâncias dos pontos até a faculdade
  for i in range(num_pontos):
    latitude, longitude = list(zip(lats, lons))[i]
    coords_ponto = (latitude, longitude)
    dist = geopy.distance.vincenty(coords_faculdade, coords_ponto).km
    distances.append(dist)
  
  return distances

In [86]:
# Gerar dataset de cada linha de ônibus
for linha in range(22, len(linhas_onibus)):
  # Pegando nome e link da linha
  linha_nome, linha_link = linhas_onibus.iloc[linha, :]
  print('*'*60)
  print('LINHA {}'.format(linha_nome))
  print('*'*60)

  # Coletando dados
  soup = get_soup(linha_link)
  pontos_onibus = get_pontos_addresses(soup)
  lats, lons = get_coords(pontos_onibus)
  dists = get_dists(len(pontos_onibus), lats, lons)

  # Criando o dataframe da linha
  df_pontos = pd.DataFrame(data=None, columns=['num_ponto', 'ponto', 'latitude', 'longitude', 'distancia_unicamp'])
  df_pontos['num_ponto'] = range(1, len(pontos_onibus)+1)
  df_pontos['ponto'] = pontos_onibus
  df_pontos['latitude'] = lats
  df_pontos['longitude'] = lons
  df_pontos['distancia_unicamp'] = dists

  # Salvando o dataframe
  linha_num = re.search(r"^[0-9]*", linha_nome).group(0)
  df_pontos.to_csv(r'linha_{}.csv'.format(linha_num), index=False
                   )

************************************************************
LINHA 400 (Rapidão Sul) Palmeira Real - Terminal Central
************************************************************
Digite a coordenada de 'Museu Major José Levy Sobrinho' (x, y): -22.565439554677592, -47.40223138902325
Digite a coordenada de 'Avenida Tenente Aviador Eli Horta Barbosa, 1-63' (x, y): -22.57351595307891, -47.413904029897566
Digite a coordenada de 'Avenida Comendador Agostinho Prada, 3045-3201 (Rapidão)' (x, y): -22.583273277108276, -47.43077591523882
Digite a coordenada de 'Rua Octávio Ribeiro, 82-130 (Rapidão)' (x, y): -22.58673716066072, -47.43462781934347
Digite a coordenada de 'Rua João Maesi, 237 (Rapidão)' (x, y): -22.592120271825166, -47.43572126901734
Digite a coordenada de 'Rua Dom Tarcísio Ariovaldo Do Amaral, 1027-1073 (Rapidão)' (x, y): -22.59212079980574, -47.43096693746713
Digite a coordenada de 'Rua José Alberto Campanini, 1720 (Rapidão)' (x, y): -22.59804806971839, -47.42055695989183
Digite a 

In [ ]:
'''
************************************************************
LINHA 01 Jd. Campo Verde / G. Piccinini x Shopping/Atacadão
************************************************************
Digite a coordenada de 'Shopping Center Limeira - Atacadão B/C' (x, y): -22.533111140137375, -47.38589692123141
Digite a coordenada de 'Museu Major José Levy Sobrinho' (x, y): -22.565439554677592, -47.40223138902325
Digite a coordenada de 'Rua Baía, 278-364' (x, y): -22.579095154169337, -47.40571906043639
Digite a coordenada de 'Rua Baía, 538' (x, y): -22.5793954797264, -47.403170644416065
Digite a coordenada de 'Rua Doutor Wiliam Silva, 272-338' (x, y): -22.587341911402365, -47.405741895265045
Digite a coordenada de 'Rua Joaquim Pompéu, 485' (x, y): -22.58979820938725, -47.406394650217855
Digite a coordenada de 'Ponto Campo Verde' (x, y): -22.603162776273155, -47.38954455465586
************************************************************
LINHA 02 Olga Veroni x Geada
************************************************************
Digite a coordenada de 'Ponto Final Geada' (x, y): -22.54534799380873, -47.45448054085058
Digite a coordenada de 'Avenida Doutor João Amaral Gama, 391 (Rapidão)' (x, y): -22.558555444439218, -47.442630355315096
Digite a coordenada de 'Rua Professor Rui Côrte Brilho, 816-880' (x, y): -22.561134208423862, -47.43595336973164
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 975-991' (x, y): -22.559066875118823, -47.4229054440875
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 685' (x, y): -22.558620803921194, -47.42013176091486
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 131-181' (x, y): -22.560728231386136, -47.41555660069769
Digite a coordenada de 'Rua Capitão Francisco Sérgio De Toledo, 145-207' (x, y): -22.558197007552785, -47.412828403187596
Digite a coordenada de 'Nosso Clube' (x, y): -22.558720436605455, -47.409808033403905
Digite a coordenada de 'Escola Brasil - Praça José Bonifácio B/C' (x, y): -22.561011950736074, -47.40396744752847
Digite a coordenada de 'Inss' (x, y): -22.569341176363963, -47.400102804456175
Digite a coordenada de 'Avenida Campinas, 1152  (Rapidão)' (x, y): -22.577078622765704, -47.39148226761585
Digite a coordenada de 'Avenida Marechal Artur Da Costa E Silva, 111' (x, y): -22.581854394170282, -47.38515237650475
Digite a coordenada de 'Rua Pedro Elías, 546  (Rapidão)' (x, y): -22.579961253882317, -47.381879508771206
Digite a coordenada de 'Ponto Final Olga Veroni' (x, y): -22.581196104338872, -47.36869089689841
************************************************************
LINHA 03 Nova Limeira x N. Sra. das Dores
************************************************************
Digite a coordenada de 'Rua Professor Rui Côrte Brilho, 180-260' (x, y): -22.565706782726892, -47.438840529780876
Digite a coordenada de 'Rua Professor Rui Côrte Brilho, 816-880' (x, y): -22.561352178130917, -47.43596459977636
Digite a coordenada de 'Museu Major José Levy Sobrinho B/C (Rapidão)' (x, y): -22.565484879341504, -47.40211036070088
Digite a coordenada de 'Quadra Da União' (x, y): -22.56149859301894, -47.396966160724496
Digite a coordenada de 'Qualicytrus' (x, y): -22.559117337095966, -47.38976799480118
************************************************************
LINHA 04 Olga Veroni x Jardim Vanessa
************************************************************
Digite a coordenada de 'Ponto Final Vanessa' (x, y): -22.540353763575293, -47.42502573467113
Digite a coordenada de 'Escola Brasil - Praça José Bonifácio B/C' (x, y): -22.56061537605523, -47.40419358435726
Digite a coordenada de 'Quadra Da União' (x, y): -22.56149859301894, -47.396966160724496
Digite a coordenada de 'Ponto Nº741' (x, y): -22.563320444552577, -47.39155022149111
Digite a coordenada de 'Avenida Assis Brasil, 610  (Rapidão)' (x, y): -22.567265823927244, -47.390318127105076
Digite a coordenada de 'Rua Vereador Samuel Berto, 1090  (Rapidão)' (x, y): -22.56934567025463, -47.38753371188035
Digite a coordenada de 'Rua João Batistela, 351  (Rapidão)' (x, y): -22.571798902447632, -47.38553314393396
Digite a coordenada de 'Rua Pedro Elías, 546  (Rapidão)' (x, y): -22.579971461108798, -47.38187292767528
Digite a coordenada de 'Via Luís Varga, 6856-6918  (Rapidão)' (x, y): -22.578884433223408, -47.377038222653034
Digite a coordenada de 'Rua Presidente Manoel Joaquim Albuquerque Lins, 364-410' (x, y): -22.580826756928012, -47.37595241541918
Digite a coordenada de 'Rua Presidente Manoel Joaquim Albuquerque Lins, 642-690' (x, y): -22.580325707528367, -47.373506078067464
Digite a coordenada de 'Ponto Final Olga Veroni' (x, y): -22.581196104338872, -47.36869089689841
************************************************************
LINHA 06 Campo Novo x N. Sra. das Dores
************************************************************
Digite a coordenada de 'Avenida Gustavo Battistela, 706' (x, y): -22.597077506362975, -47.369815078418526
Digite a coordenada de 'Avenida Gustavo Battistela, 926' (x, y): -22.59929490874741, -47.36947162069703
Digite a coordenada de 'Rua Olívia P. B Pinho, 104' (x, y): -22.589627980897358, -47.37226183781522
Digite a coordenada de 'Rua Olívia P. B Pinho, 190-288' (x, y): -22.589164199194943, -47.37203482476265
Digite a coordenada de 'Via Luís Varga, 862 (Rapidão)' (x, y): -22.58396569407025, -47.3753970646546
Digite a coordenada de 'Via Luís Varga, 7325-7375  (Rapidão)' (x, y): -22.578725025805046, -47.37688560683955
Digite a coordenada de 'Avenida Tenente Aviador Eli Horta Barbosa, 1-63' (x, y): -22.573507753743584, -47.413885179526666
Digite a coordenada de 'Rua Ulda Guilhermina Schinoor Heldt' (x, y): -22.572962133679646, -47.45139098764576
Digite a coordenada de 'Rua Luís Bueno De Miranda, 559-637' (x, y): -22.576240984260828, -47.457598220510484
Digite a coordenada de 'Rua João Antônio Soares Neto, 434-464' (x, y): -22.57305921632801, -47.45204681806234
Digite a coordenada de 'Rua João Antônio Soares Neto, 592' (x, y): -22.57168349654549, -47.45279987977735
************************************************************
LINHA 08B Atacadão x Terminal
************************************************************
Digite a coordenada de 'Shopping Center Limeira - Atacadão C/B' (x, y): -22.533111140137375, -47.38589692123141
Digite a coordenada de 'Shopping Center Limeira - Atacadão B/C' (x, y): -22.533111140137375, -47.38589692123141
Digite a coordenada de 'Rua Moisés Paulino Da Cunha, 327-337' (x, y): -22.565908759611087, -47.37614056950166
Digite a coordenada de 'Rua Doutor Ademar Pereira De Barros, 697-765  (Rapidão)' (x, y): -22.57143879353656, -47.37424973972138
Digite a coordenada de 'Via Luís Varga, 6856-6918  (Rapidão)' (x, y): -22.57822842542831, -47.37722799698128
Digite a coordenada de 'Rua Pedro Elías 765  (Rapidão)' (x, y): -22.57943279150629, -47.37989197064324
Digite a coordenada de 'Rua Pedro Elías, 141-147  (Rapidão)' (x, y): -22.578659503561827, -47.37647515730939
Digite a coordenada de 'Avenida Marechal Artur Da Costa E Silva, 942' (x, y): -22.58582615352546, -47.37906785165745
Digite a coordenada de 'Rua Olívia P. B Pinho, 63-127' (x, y): -22.589667233425953, -47.37225301058662
Digite a coordenada de 'Rua José Noberto De Carvalho' (x, y): -22.600163521346552, -47.384799211051934
Digite a coordenada de 'Rua João Daniel Dos Santos' (x, y): -22.60314454060133, -47.3850107672932
Digite a coordenada de 'Avenida Campinas, 141-239 (Rapidão)' (x, y): -22.569984565938082, -47.397862966147216
************************************************************
LINHA 08C Santina - Parque Hipólito via Santa Casa
************************************************************
Digite a coordenada de 'Terminal Parque Hipolito' (x, y): -22.566720476594224, -47.37013678604166
Digite a coordenada de 'Rua Doutor Ademar Pereira De Barros, 697-765  (Rapidão)' (x, y): -22.571487264811793, -47.37422361108311
Digite a coordenada de 'Via Luís Varga, 6856-6918  (Rapidão)' (x, y): -22.57824643615471, -47.37719895074995
Digite a coordenada de 'Rua Pedro Elías 765  (Rapidão)' (x, y): -22.57943279150629, -47.37989197064324
Digite a coordenada de 'Rua João Batistela, 342  (Rapidão)' (x, y): -22.571785091439367, -47.385454602742634
Digite a coordenada de 'Rua Vereador Samuel Berto, 1025-1099  (Rapidão)' (x, y): -22.569092195869104, -47.38762607733455
Digite a coordenada de 'Avenida Assis Brasil, 589  (Rapidão)' (x, y): -22.567286552250078, -47.39053815860938
Digite a coordenada de 'Avenida Assis Brasil, 423-499  (Rapidão)' (x, y): -22.56757517802494, -47.39213506719198
Digite a coordenada de 'Museu Major José Levy Sobrinho' (x, y): -22.565482060847764, -47.40206587155135
Digite a coordenada de 'Rua Baía, 278-364' (x, y): -22.579164906361118, -47.40570176534706
Digite a coordenada de 'Rua Baía, 538' (x, y): -22.579405279348784, -47.40317015821637
Digite a coordenada de 'Rua Baía, 688' (x, y): -22.579555003875733, -47.40191103656098
Digite a coordenada de 'Ponto Final Santina' (x, y): -22.599632477316757, -47.39342939065967
************************************************************
LINHA 09 Tatu (Parronchi) x Terminal
************************************************************
Digite a coordenada de 'Cerâmica Unigrês' (x, y): -22.603361493647267, -47.488482889747736
Digite a coordenada de 'Museu Major José Levy Sobrinho B/C (Rapidão)' (x, y): -22.565482060847764, -47.40206587155135
************************************************************
LINHA 10 Terminal x Pinhal (Pires)
************************************************************
Digite a coordenada de 'Ponto Final Pinhal' (x, y): -22.55013682878644, -47.216222832990546
Digite a coordenada de 'Ponto Da Parada Trevo' (x, y): -22.569041336955458, -47.17399215227436
Digite a coordenada de 'Ponto De Parada Caixa D'Agua' (x, y): -22.45138465383065, -46.844253724723984
Digite a coordenada de 'Condomínio Das Hortencias' (x, y): -22.527953039104773, -47.31746213920673
Digite a coordenada de 'Ponto Pesqueiro Lago Grande' (x, y): -22.51722435568518, -47.319430981735884
Digite a coordenada de 'Recanto Dos Carajas' (x, y): -22.538321705079642, -47.279331284984146
Digite a coordenada de 'Cherry' (x, y): -22.55909896899743, -47.39017462485447
Digite a coordenada de 'Central De Proteção' (x, y): -22.559338610925533, -47.39344070536366
************************************************************
LINHA 11 Nobre Ville x Village
************************************************************
Digite a coordenada de 'Museu Major José Levy Sobrinho' (x, y): -22.565482060847764, -47.40206587155135
Digite a coordenada de 'Rua Baía, 278-364' (x, y): -22.579164906361118, -47.40570176534706
Digite a coordenada de 'Rua Baía, 538' (x, y): -22.579405279348784, -47.40317015821637
Digite a coordenada de 'Rua Baía, 688' (x, y): -22.579555003875733, -47.40191103656098
Digite a coordenada de 'Clube Do Galo' (x, y): -22.58589836261789, -47.40182860200851
Digite a coordenada de 'Vila Antônio Cruãnes Filho' (x, y): -22.585296466835988, -47.397252277842234
Digite a coordenada de 'Rua Lauro Camargo Silveira, 68-130' (x, y): -22.601289669776346, -47.41869360900967
Digite a coordenada de 'Rua Lauro Camargo Silveira, 198-260' (x, y): -22.602809181346263, -47.41773512322814
Digite a coordenada de 'Rua Lauro Camargo Silveira, 460-524' (x, y): -22.603938966611768, -47.41729637486011
Digite a coordenada de 'Rua Lauro Camargo Silveira, 723-787' (x, y): -22.605761498259362, -47.4164063545446
Digite a coordenada de 'Rua Lauro Camargo Silveira, 1294-1338' (x, y): -22.612292241045356, -47.41464178064791
Digite a coordenada de 'Ponto Final Odecio Degan' (x, y): -22.61348469322839, -47.41544428007784
Digite a coordenada de 'Rua Vereador Osvaldo Ferreira Dos Santos, 124' (x, y): -22.607561470434163, -47.419420449529646
Digite a coordenada de 'Rua Vereador Osvaldo Ferreira Dos Santos, 316' (x, y): -22.606796384255137, -47.421116291931774
Digite a coordenada de 'Avenida Doutor Lauro Corrêa Da Silva, 928-994 (Rapidão)' (x, y): -22.608061522865924, -47.422194693893786
Digite a coordenada de 'Ponto Final' (x, y): -22.61001261014933, -47.42261657075821
************************************************************
LINHA 11B Lagoa Nova x Terminal
************************************************************
Digite a coordenada de 'Ponto Final Lagoa Nova' (x, y): -22.61669789085583, -47.41288370093281
Digite a coordenada de 'Rua Lauro Camargo Silveira, 1293-1337' (x, y): -22.61236352073077, -47.41462729158096
Digite a coordenada de 'Rua Lauro Camargo Silveira, 1053-1117' (x, y): -22.6101928586952, -47.415074032082906
Digite a coordenada de 'Rua Lauro Camargo Silveira, 789-853' (x, y): -22.606186414376424, -47.41626583827723
Digite a coordenada de 'Rua Vereador Osvaldo Ferreira Dos Santos, 124' (x, y): -22.60757115649052, -47.41940582018568
Digite a coordenada de 'Rua Vereador Osvaldo Ferreira Dos Santos, 316' (x, y): -22.606820532311335, -47.421080733146525
Digite a coordenada de 'Rua Dorival Faver, 322-360 (Rapidão)' (x, y): -22.604347778890425, -47.42374419120848
Digite a coordenada de 'Rua Dom Tarcísio Ariovaldo Do Amaral, 1070 (Rapidão)' (x, y): -22.592138440668677, -47.43093496177005
Digite a coordenada de 'Rua Octávio Ribeiro, 89 (Rapidão)' (x, y): -22.586489535195685, -47.43450511048656
Digite a coordenada de 'Avenida Um A, 1387' (x, y): -22.59109158177076, -47.42832792910649
Digite a coordenada de 'Avenida Um A, 905' (x, y): -22.590121094170655, -47.43039125418448
Digite a coordenada de 'Museu Major José Levy Sobrinho B/C (Rapidão)' (x, y): -22.565482060847764, -47.40206587155135
************************************************************
LINHA 12 Santa Eulália x N. Sra. das Dores (via Santa Casa)
************************************************************
Digite a coordenada de 'Museu Major José Levy Sobrinho B/C (Rapidão)' (x, y): -22.565482060847764, -47.40206587155135
Digite a coordenada de 'Catedral N.Sr. Das Dores (Rapidão)' (x, y): -22.56157386008471, -47.402000260308085
Digite a coordenada de 'Escola Brasil - Praça José Bonifácio C/B' (x, y): -22.561011950736074, -47.40396744752847
Digite a coordenada de 'Avenida Ana Carolina De Barros Levi 124' (x, y): -22.559243854527736, -47.407287711143034
Digite a coordenada de 'Avenida Ana Carolina De Barros Levi, 260-450 (Rapidão)' (x, y): -22.55904821509278, -47.40834107095815
Digite a coordenada de 'Avenida Ana Carolina De Barros Levi, 458-494' (x, y): -22.558250582857216, -47.41001558654577
Digite a coordenada de 'Rua Capitão Manoel Ferraz Camargo, 122 (Rapidão)' (x, y): -22.556137010690858, -47.413831457709364
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 726' (x, y): -22.558518975285452, -47.42062986042478
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 930' (x, y): -22.55900715038987, -47.42268512660228
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 1212' (x, y): -22.559541125575418, -47.42523536540395
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 1518-1560' (x, y): -22.560779043320487, -47.427648567200386
Digite a coordenada de 'Rua Osvaldo Bertoline, 152' (x, y): -22.560237292325944, -47.429629222555526
Digite a coordenada de 'Rua Osvaldo Bertoline, 294-346' (x, y): -22.55950706442501, -47.43082261101142
Digite a coordenada de 'Rua Osvaldo Bertoline, 512' (x, y): -22.55845662648252, -47.43266718058858
Digite a coordenada de 'Rua Carlos Guilherme Schnoor, 180-380' (x, y): -22.557462566803064, -47.43537359193769
Digite a coordenada de 'Avenida Jacobe Cover, 1015' (x, y): -22.559106737960615, -47.44297572634121
Digite a coordenada de 'Avenida Jacobe Cover, 835' (x, y): -22.560767824915338, -47.44316460460508
Digite a coordenada de 'Avenida Jacobe Cover, 179-287' (x, y): -22.563935668892587, -47.44337106570962
************************************************************
LINHA 13 B. Grotta x Geada (via Santa Casa)
************************************************************
Digite a coordenada de 'Avenida Luís Cavinato, 394-408' (x, y): -22.596821609431665, -47.407929806177236
Digite a coordenada de 'Rua Túlio Taques De Lemos, 185-245' (x, y): -22.593039144584573, -47.40680906097436
Digite a coordenada de 'Rua Joaquim Pompéu, 387-451' (x, y): -22.58910638647184, -47.40607002663493
Digite a coordenada de 'Rua Doutor Wiliam Silva, 271-337' (x, y): -22.587402149022424, -47.40574332220587
Digite a coordenada de 'Rua Baía, 538' (x, y): -22.579405279348784, -47.40317015821637
Digite a coordenada de 'Rua Baía, 688' (x, y): -22.579555003875733, -47.40191103656098
Digite a coordenada de 'Funerária Bom Pastor' (x, y): -22.552415995525056, -47.37949040734621
Digite a coordenada de 'Rua Santa Eliza, 122' (x, y): -22.5736129971867, -47.40848708917393
Digite a coordenada de 'Museu Major José Levy Sobrinho B/C (Rapidão)' (x, y): -22.565482060847764, -47.40206587155135
Digite a coordenada de 'Catedral N.Sr. Das Dores (Rapidão)' (x, y): -22.56157386008471, -47.402000260308085
Digite a coordenada de 'Rua Capitão Flamínio Ferreira, 489-585' (x, y): -22.567526924557047, -47.4094484448596
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 101-167' (x, y): -22.560701256587084, -47.415506424211316
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 726' (x, y): -22.558543222729284, -47.42064425539583
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 930' (x, y): -22.5590094417888, -47.42274176366197
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 1212' (x, y): -22.559541125575418, -47.42523536540395
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 1518-1560' (x, y): -22.560779043320487, -47.427648567200386
Digite a coordenada de 'Rua Osvaldo Bertoline, 152' (x, y): -22.56025720385242, -47.42961979272565
Digite a coordenada de 'Rua Osvaldo Bertoline, 294-346' (x, y): -22.55950706442501, -47.43082261101142
Digite a coordenada de 'Rua Osvaldo Bertoline, 512' (x, y): -22.55845662648252, -47.43266718058858
Digite a coordenada de 'Rua Professor Rui Côrte Brilho, 819' (x, y): -22.561216801268866, -47.43590213381577
Digite a coordenada de 'Ponto Final Geada' (x, y): -22.54534799380873, -47.45448054085058
************************************************************
LINHA 15 UNIP x Terminal (via Laranjeiras)
************************************************************
Digite a coordenada de 'Catedral N.Sr. Das Dores (Rapidão)' (x, y): -22.56157386008471, -47.402000260308085
Digite a coordenada de 'Shopping Center Limeira - Atacadão C/B' (x, y): -22.533111140137375, -47.38589692123141
Digite a coordenada de 'Unip - Universidade Paulista' (x, y): -22.544298393958066, -47.3776863664987
************************************************************
LINHA 102 Parque Hipólito x N. Sra. das Dores (via Medical)
************************************************************
Digite a coordenada de 'Rua Professor Rui Côrte Brilho, 180-260' (x, y): -22.56564041285834, -47.43891425353376
Digite a coordenada de 'Rua Professor Rui Côrte Brilho, 816-880' (x, y): -22.561309140844312, -47.435968875963894
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 975-991' (x, y): -22.55915604605529, -47.42288398493309
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 685' (x, y): -22.55859193513618, -47.42015416136711
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 131-181' (x, y): -22.560786383208978, -47.41556288999053
Digite a coordenada de 'Rua Capitão Francisco Sérgio De Toledo, 145-207' (x, y): -22.558166810684092, -47.412796739235525
Digite a coordenada de 'Nosso Clube' (x, y): -22.558720436605455, -47.409808033403905
Digite a coordenada de 'Escola Brasil - Praça José Bonifácio B/C' (x, y): -22.561011950736074, -47.40396744752847
Digite a coordenada de 'Paraíso Dos Animais' (x, y): -22.57063507415941, -47.39858487448864
Digite a coordenada de 'Rua Oto Frederico Bürger, 366  (Rapidão)' (x, y): -22.57311315203243, -47.38094964774555
Digite a coordenada de 'Rua Presidente Getúlio Vargas Cidade De Limeira, 420' (x, y): -22.57226405591138, -47.37524215870534
Digite a coordenada de 'Rua Placidina Ferreira Braga, 420' (x, y): -22.570290177420716, -47.37563854160362
Digite a coordenada de 'Terminal Parque Hipolito' (x, y): -22.566720476594224, -47.37013678604166
************************************************************
LINHA 103 Odécio Degan x Parque Hipólito (via Santa Casa)
************************************************************
Digite a coordenada de 'Ponto Final Odécio Degan' (x, y): -22.61348469322839, -47.41544428007784
Digite a coordenada de 'Rua Três, 4085-4215 (Rapidão)' (x, y): -22.6073941730483, -47.40848502307874
Digite a coordenada de 'Ponto Final Linha Sta.Eulalia' (x, y): -22.602999139430082, -47.40612199372663
Digite a coordenada de 'Rua Antonio Fortes, 291-343' (x, y): -22.600750577990787, -47.40884042298899
Digite a coordenada de 'Rua Ana Tumenas Tolle, 3236-3290 (Rapidão)' (x, y): -22.60519584345984, -47.406847268555794
Digite a coordenada de 'Rua Ana Tumenas Tolle, 3446-3494 (Rapidão)' (x, y): -22.606186745981933, -47.40442238624962
Digite a coordenada de 'Avenida Fausto Esteves Dos Santos, 1425-1491 (Rapidão)' (x, y): -22.60149797048027, -47.40159392879489
Digite a coordenada de 'Funerária Bom Pastor' (x, y): -22.552415995525056, -47.37949040734621
Digite a coordenada de 'Rua Santa Eliza, 122' (x, y): -22.5736129971867, -47.40848708917393
Digite a coordenada de 'Museu Major José Levy Sobrinho B/C (Rapidão)' (x, y): -22.565482060847764, -47.40206587155135
Digite a coordenada de 'Paraíso Dos Animais' (x, y): -22.57063507415941, -47.39858487448864
Digite a coordenada de 'Avenida Campinas, 1152  (Rapidão)' (x, y): -22.577078622765704, -47.39148226761585
Digite a coordenada de 'Avenida Marechal Artur Da Costa E Silva, 111' (x, y): -22.581904001871347, -47.38515896772426
Digite a coordenada de 'Rua Pedro Elías, 546  (Rapidão)' (x, y): -22.579980144615405, -47.38187649120758
Digite a coordenada de 'Rua Mario Macedo Soares, 8  (Rapidão)' (x, y): -22.57267718801858, -47.37177504858215
Digite a coordenada de 'Rua Mario Macedo Soares, 486-558  (Rapidão)' (x, y): -22.571733129100597, -47.36984121300883
Digite a coordenada de 'Rua Encarnação Carrasco Arias, 511-647  (Rapidão)' (x, y): -22.571688924060823, -47.368823636049704
Digite a coordenada de 'Rua Joaquim Carlos Wissel, 439  (Rapidão)' (x, y): -22.57083918930491, -47.367831584026305
Digite a coordenada de 'Rua Joaquim Carlos Wissel, 91-147  (Rapidão)' (x, y): -22.567691879962304, -47.36770898393665
Digite a coordenada de 'Terminal Parque Hipolito' (x, y): -22.566720476594224, -47.37013678604166
************************************************************
LINHA 104 Belinha Ometto x Jardim Anhanguera (via Sta.Medical)
************************************************************
Digite a coordenada de 'Ponto Final  B.Ometto' (x, y): -22.55236050392411, -47.45026156851256
Digite a coordenada de 'Avenida Jacobe Cover, 1015' (x, y): -22.55910275178576, -47.442986802891284
Digite a coordenada de 'Avenida Jacobe Cover, 835' (x, y): -22.56075414713605, -47.443131165571174
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 1921-1991' (x, y): -22.56098110163366, -47.431093011132354
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 1609' (x, y): -22.561477105230733, -47.428159083922054
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 975-991' (x, y): -22.559182816068365, -47.42293757032946
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 685' (x, y): -22.55859193513618, -47.42015416136711
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 131-181' (x, y): -22.560786383208978, -47.41556288999053
Digite a coordenada de 'Rua Capitão Francisco Sérgio De Toledo, 145-207' (x, y): -22.558166810684092, -47.412796739235525
Digite a coordenada de 'Nosso Clube' (x, y): -22.558720436605455, -47.409808033403905
Digite a coordenada de 'Escola Brasil - Praça José Bonifácio B/C' (x, y): -22.561011950736074, -47.40396744752847
Digite a coordenada de 'Paraíso Dos Animais' (x, y): -22.57063507415941, -47.39858487448864
Digite a coordenada de 'Rua Olívia P. B Pinho, 104' (x, y): -22.589627980897358, -47.37226183781522
Digite a coordenada de 'Rua Olívia P. B Pinho, 190-288' (x, y): -22.589164199194943, -47.37203482476265
Digite a coordenada de 'Avenida Marechal Artur Da Costa E Silva, 1225' (x, y): -22.586892569087905, -47.37629728411523
Digite a coordenada de 'Avenida Marechal Artur Da Costa E Silva, 889-933' (x, y): -22.585533586099142, -47.379290765962544
************************************************************
LINHA 105 UNIP x N. Sra. das Dores (via Shopping)
************************************************************
Digite a coordenada de 'Rua Carlos Guilherme Schnoor, 237' (x, y): -22.556881197346744, -47.4368269859418
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 975-991' (x, y): -22.559182816068365, -47.42293757032946
Digite a coordenada de 'Norte Avenida Doutor Fabrício Vampré, 685' (x, y): -22.55859193513618, -47.42015416136711
Digite a coordenada de 'Escola Brasil - Praça José Bonifácio B/C' (x, y): -22.561011950736074, -47.40396744752847
Digite a coordenada de 'Quadra Da União' (x, y): -22.56149859301894, -47.396966160724496
Digite a coordenada de 'Qualicytrus' (x, y): -22.559117337095966, -47.38976799480118
Digite a coordenada de 'Rua Sebastiana Guidote Campos, 519-755' (x, y): -22.5352104320332, -47.393569154148445
Digite a coordenada de 'Shopping Center Limeira - Atacadão C/B' (x, y): -22.533111140137375, -47.38589692123141
Digite a coordenada de 'Unip - Universidade Paulista' (x, y): -22.544298393958066, -47.3776863664987
************************************************************
LINHA 107 Odécio Degan x Atacadão (via Santa Casa)
************************************************************
Digite a coordenada de 'Shopping Center Limeira - Atacadão C/B' (x, y): -22.533111140137375, -47.38589692123141
Digite a coordenada de 'Shopping Center Limeira - Atacadão B/C' (x, y): -22.533111140137375, -47.38589692123141
Digite a coordenada de 'Avenida Laranjeiras, 29-279' (x, y): -22.556329474264146, -47.3983617339372
Digite a coordenada de 'Escola Brasil - Praça José Bonifácio B/C' (x, y): -22.561011950736074, -47.40396744752847
Digite a coordenada de 'Inss' (x, y): -22.569341176363963, -47.400102804456175
Digite a coordenada de 'Rua Baía, 278-364' (x, y): -22.579164906361118, -47.40570176534706
Digite a coordenada de 'Rua Baía, 538' (x, y): -22.5793954797264, -47.403170644416065
Digite a coordenada de 'Rua Baía, 688' (x, y): -22.579555003875733, -47.40191103656098
Digite a coordenada de 'Rua Doutor Wiliam Silva, 272-338' (x, y): -22.587416360712517, -47.405740349124784
Digite a coordenada de 'Rua Joaquim Pompéu, 485' (x, y): -22.589815291140397, -47.40639133517659
Digite a coordenada de 'Rua Túlio Taques De Lemos, 198' (x, y): -22.593225542290835, -47.406851046927166
Digite a coordenada de 'Rua Jorge Salibe Sobrinho, 300' (x, y): -22.595720764111352, -47.40846720107655
Digite a coordenada de 'Rua Três, 4086-4216 (Rapidão)' (x, y): -22.607636425429487, -47.40858169404741
Digite a coordenada de 'Rua Ana Tumenas Tolle, 96 (Rapidão)' (x, y): -22.605364601185745, -47.40640092572371
Digite a coordenada de 'Antonio Ferrari, 1034-1130' (x, y): -22.61583368445663, -47.41079646413807
Digite a coordenada de 'Ponto Final Odécio Degan' (x, y): -22.61348469322839, -47.41544428007784
************************************************************
LINHA 114 Lagoa Nova x Atacadão (via Shopping)
************************************************************
Digite a coordenada de 'Shopping Center Limeira - Atacadão C/B' (x, y): -22.533111140137375, -47.38589692123141
Digite a coordenada de 'Shopping Center Limeira - Atacadão B/C' (x, y): -22.533111140137375, -47.38589692123141
Digite a coordenada de 'Rua Sebastiana Guidote Campos, 520-756' (x, y): -22.535184828122883, -47.39354619993704
Digite a coordenada de 'Escola Brasil - Praça José Bonifácio B/C' (x, y): -22.561011950736074, -47.40396744752847
Digite a coordenada de 'Museu Major José Levy Sobrinho' (x, y): -22.565439554677592, -47.40223138902325
Digite a coordenada de 'Rua Baía, 291' (x, y): -22.579185520217507, -47.40550273140147
Digite a coordenada de 'Rua Lauro Camargo Silveira, 68-130' (x, y): -22.60128404756533, -47.41869417354527
Digite a coordenada de 'Rua Lauro Camargo Silveira, 198-260' (x, y): -22.60281006184232, -47.41772937337915
Digite a coordenada de 'Rua Lauro Camargo Silveira, 460-524' (x, y): -22.603844949096434, -47.41721446312992
Digite a coordenada de 'Rua Lauro Camargo Silveira, 723-787' (x, y): -22.605725110526926, -47.41639531366791
Digite a coordenada de 'Rua Lauro Camargo Silveira, 1294-1338' (x, y): -22.61237459330822, -47.41464689857391
Digite a coordenada de 'Ponto Final Lagoa Nova' (x, y): -22.61669789085583, -47.41288370093281
************************************************************
LINHA 115 Boa Esperança x Terminal
************************************************************
Digite a coordenada de 'Mega' (x, y): -22.566670741636795, -47.380531763676125
Digite a coordenada de 'Rua Placidina Ferreira Braga, 164  (Rapidão)' (x, y): -22.571666786289722, -47.37772191802971
Digite a coordenada de 'Rua Placidina Ferreira Braga, 420' (x, y): -22.57030162736904, -47.375661077712905
Digite a coordenada de 'Rua Moisés Paulino Da Cunha, 327' (x, y): -22.56592571984985, -47.37616041482262
Digite a coordenada de 'Rua Manoel Queirós, 22-144' (x, y): -22.560838624818114, -47.37996745884866
************************************************************
LINHA 124 São Lourenço x Terminal (via Santa Casa)
************************************************************
Digite a coordenada de 'Parada E Ponto Final Linha S.Lourenço  (Rapidão)' (x, y): -22.61283034095905, -47.42371605899512
Digite a coordenada de 'Rua Vereador Osvaldo Ferreira Dos Santos, 316' (x, y): -22.606805191467707, -47.42108750117579
Digite a coordenada de 'Rua Dorival Faver, 322-360 (Rapidão)' (x, y): -22.60435134762837, -47.42373341151006
Digite a coordenada de 'Rua José Pintarelli, 142-186 (Rapidão)' (x, y): -22.603066507243916, -47.4236744956645
Digite a coordenada de 'Rua José Pintarelli, 282-330 (Rapidão)' (x, y): -22.601918995141858, -47.42321555611858
Digite a coordenada de 'Rua José Pintarelli, 530 (Rapidão)' (x, y): -22.599797720595593, -47.42255814908055
Digite a coordenada de 'Rua Divanir Gazetta, 181-249 (Rapidão)' (x, y): -22.590167851202928, -47.43188290085696
Digite a coordenada de 'Rua José Alberto Campanini, 1695-1749 (Rapidão)' (x, y): -22.597768133991398, -47.420529541822894
Digite a coordenada de 'Rua Dom Tarcísio Ariovaldo Do Amaral, 1070 (Rapidão)' (x, y): -22.592116276481537, -47.43098013231621
Digite a coordenada de 'Rua Octávio Ribeiro, 89 (Rapidão)' (x, y): -22.58648892090031, -47.43450708225559
Digite a coordenada de 'Avenida Um A, 1387' (x, y): -22.59109158177076, -47.42832792910649
Digite a coordenada de 'Avenida Um A, 905' (x, y): -22.590123317999183, -47.43037858037432
Digite a coordenada de 'Rua Baía, 278-364' (x, y): -22.579095154169337, -47.40571906043639
Digite a coordenada de 'Rua Baía, 538' (x, y): -22.5793954797264, -47.403170644416065
Digite a coordenada de 'Rua Baía, 688' (x, y): -22.579555003875733, -47.40191103656098
Digite a coordenada de 'Funerária Bom Pastor' (x, y): -22.552415995525056, -47.37949040734621
Digite a coordenada de 'Rua Santa Eliza, 122' (x, y): -22.5736129971867, -47.40848708917393
Digite a coordenada de 'Museu Major José Levy Sobrinho B/C (Rapidão)' (x, y): -22.565439554677592, -47.40223138902325
************************************************************
LINHA 400 (Rapidão Sul) Palmeira Real - Terminal Central
************************************************************
Digite a coordenada de 'Museu Major José Levy Sobrinho' (x, y): -22.565439554677592, -47.40223138902325
Digite a coordenada de 'Avenida Tenente Aviador Eli Horta Barbosa, 1-63' (x, y): -22.57351595307891, -47.413904029897566
Digite a coordenada de 'Avenida Comendador Agostinho Prada, 3045-3201 (Rapidão)' (x, y): -22.583273277108276, -47.43077591523882
Digite a coordenada de 'Rua Octávio Ribeiro, 82-130 (Rapidão)' (x, y): -22.58673716066072, -47.43462781934347
Digite a coordenada de 'Rua João Maesi, 237 (Rapidão)' (x, y): -22.592120271825166, -47.43572126901734
Digite a coordenada de 'Rua Dom Tarcísio Ariovaldo Do Amaral, 1027-1073 (Rapidão)' (x, y): -22.59212079980574, -47.43096693746713
Digite a coordenada de 'Rua José Alberto Campanini, 1720 (Rapidão)' (x, y): -22.59804806971839, -47.42055695989183
Digite a coordenada de 'Rua Divanir Gazetta, 210 (Rapidão)' (x, y): -22.59016624081033, -47.43190043444842
Digite a coordenada de 'Rua José Pintarelli, 25 (Rapidão)' (x, y): -22.604216524232527, -47.42409981778979
Digite a coordenada de 'Rua José Pintarelli, 315 (Rapidão)' (x, y): -22.601627180804996, -47.42312748184898
Digite a coordenada de 'Rua José Pintarelli, 153 (Rapidão)' (x, y): -22.603080925405674, -47.42368982577805
Digite a coordenada de 'Rua Dorival Faver, 321-359 (Rapidão)' (x, y): -22.604354819990515, -47.4237320658747
Digite a coordenada de 'Avenida Doutor Lauro Corrêa Da Silva, 928-994 (Rapidão)' (x, y): Avenida Doutor Lauro Corrêa Da Silva, 928-994
'''

## Teste de Visualização

In [111]:
df = pd.read_csv('linha_103.csv')
df.head()

,num_ponto,ponto,latitude,longitude,distancia_unicamp
0,1,Ponto Final Odécio Degan,-22.613485,-47.415444,5.784448
1,2,"Avenida Antônio Conselheiro, 538",-22.610988,-47.411791,5.585064
2,3,"Avenida Antônio Conselheiro, 746",-22.610988,-47.411791,5.585064
3,4,"Rua Chico Mendes, 1586-1638",-22.611197,-47.409985,5.652658
4,5,"Rua 10 De Dezembro, 127",-22.610575,-47.407206,5.667523


In [112]:
import folium
from pprint import pprint

# Criação do mapa
map = folium.Map(location=[-22.56508562812944, -47.40434392781407], tiles='cartodbpositron', zoom_start=20)

# Tooltip
tooltip_acomodacao = "Clique para ver o nome do ponto"
colors = ['blue', 'red']

# Markers
for i in range(len(df)):
  folium.Marker(df.iloc[i, 2:-1], popup=f"<i>Ponto {df['num_ponto'].iloc[i]}: {df['ponto'].iloc[i]}</i>", tooltip=tooltip_acomodacao,icon=folium.Icon(color=colors[i%2], icon="bus", prefix='fa')).add_to(map)

map

In [101]:
df.iloc[14]

num_ponto                           15
ponto                Praça A. Mesquita
latitude                      -23.5407
longitude                     -46.6425
distancia_unicamp              134.782
Name: 14, dtype: object